# Reference Light Curves of SN

#### The basic plan

The set of quantities related to supernovae used in Twinkles are in a table called TwinkSN
on a computer at UW. This code in the `Monitor` package accesses this data to have a complete
characterization of the SN truth parameters.

It then creates an instance of `lsst.sims.catUtils.supernovae.SNObject` using these values.
Since SNObject has methods that estimate flux, and estimated flux uncertainty given 
parameters that characterize the observation, we can use this to find the light curves of
the supernovae. The light curves have two main quantities, the measured flux and uncertainty
on the measured flux. These quantities depend on the properties of the astrophysical objects,
and the properties of the observation. By properties of observation, we mean site/Hardware
properties and the properties of the sky at the time of the observation. The most important of
these are the bandpasses (transmissions) and fivesigma depth (obtained from OpSim). 

Thus,  `monitor.RefLightCuves` must be passed these quantities in order to obtain the light curve.
However, for analyzing the Twinkles run, where we have a clearer idea of both the bandpasses (LSST)
and the properties of the observation (from a selection of OpSim visits), we have a simple method to
set this up:
`monitor.RefLightCurve.fromTwinklesData`.

#### Imports and Setup

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import os

In [3]:
import numpy as np
import pandas as pd

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

/usr/local/manual/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [5]:
# Import from monitor
from desc.monitor import RefLightCurves
import desc.monitor as monitor

/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


## Observations

The quantities needed for the light curves are time (MJD), and fiveSigmaDepth. While this can come
from various formats, right now, there is only one that has been implemented: a  dataframe. The data directory
of the package `desc.Monitor` contains a csv file which has the necessary pointings

In [6]:
data_dir = os.path.join(os.environ['MONITOR_DIR'], 'data')
opsimCsv = os.path.join(data_dir, 'SelectedKrakenVisits.csv')
opsimdf = pd.read_csv(opsimCsv, index_col='obsHistID')
df = opsimdf[['expMJD', 'filter', 'fiveSigmaDepth']]

### Obtaining the parameters from the database

Setting up a connection to the CatSim DataBase which has parameters of the SN model.

In [7]:
import pymssql
from lsst.utils import getPackageDir
import lsst.sims.catUtils.baseCatalogModels as bcm
from lsst.daf.persistence import DbAuth

In [8]:
config = bcm.BaseCatalogConfig()
config.load(os.path.join(getPackageDir("sims_catUtils"), "config", "db.py"))

In [9]:
username = DbAuth.username(config.host, config.port)    
password = DbAuth.password(config.host, config.port)
hostname = config.host
DBConnection = pymssql.connect(user=username,
                               password=password,
                               host=hostname,
                               database=config.database,
                               port=config.port)
db = DBConnection.cursor()

#### The RefLightCurves Class

This is instantiated in the following way:

In [10]:
# The ids are obtained from Instance Catalogs
reflc = RefLightCurves(idSequence=(6144007055260714, 6144158471480362),
                       tableName='TwinkSN',
                       dbConnection=DBConnection,
                       dbCursor=db)

### Find the number of objects in the table

In [11]:
reflc.dbConnection

In [12]:
print(reflc.get_numObjects())

776620


In [13]:
reflc.get_numObjects()

776620

### Get all ids on the table

What are the IDs in the table (unique IDs)?

In [14]:
ids = reflc.allIdinTable(chunksize=None)
print(ids.astype(int).values.flatten())

[6000000005002 6000000005001 6000000005000 ..., 6001742779700 6001742827001
 6001742827000]


The astrophysical object properties can be obtained by using the following function in the form of a `pd.DataFrame`

Or if the table is too large, and you only want to get a few of the IDs

In [15]:
ids = reflc.allIdinTable(chunksize=10)
print(ids.next().astype(int).values.flatten())

[6000000005002 6000000005001 6000000005000 6000000013401 6000000013400
 6000000013403 6000000013402 6000000013700 6000000014902 6000000014903]


Get the parameters for an object

In [16]:
reflc.get_params(6144007055260714)

,redshift,snra,sndec,t0,x0,x1,c
snid,,,,,,,
6000006889903,0.0792,53.005626,-27.389538,3627.69276,0.0008,-0.286958,-0.086294


In [17]:
allParamsInIdSequence = reflc.get_params()
allParamsInIdSequence

,redshift,snra,sndec,t0,x0,x1,c
snid,,,,,,,
6000006889903,0.0792,53.005626,-27.389538,3627.692760,0.000800,-0.286958,-0.086294
6000154757305,0.0882,53.051542,-27.414282,3565.558086,0.001018,0.607759,-0.261934


In [18]:
reflcAll = RefLightCurves(tableName='TwinkSN',
                       dbConnection=DBConnection,
                       dbCursor=db)

In [19]:
# Slow because all rows
allParams = reflcAll.get_params()

In [20]:
allParams.head()

,redshift,snra,sndec,t0,x0,x1,c
snid,,,,,,,
6001163623700,0.3401,52.803079,-27.843361,1731.094090,0.000025,-1.981708,-0.043500
6000324908000,0.3249,52.797229,-27.843795,1664.942389,0.000037,0.697824,-0.140121
6000324908001,0.3249,52.797248,-27.843597,1608.347547,0.000025,-0.238670,0.053811
6000324908002,0.3249,52.797264,-27.843773,350.999770,0.000025,0.220203,0.061134
6000350930401,0.6747,52.798069,-27.846654,2128.934351,0.000005,0.052383,0.120673


The instance of the class representing the astrophysical object itself can be obtained by the following method for SN

In [21]:
sn = reflc.astro_object(idValue=6144007055260714)

Following the usual methods in `sims.catUtils.supernovae.SNObject` the properties of this SN can be seen using

## Calculating Light Curves

In [22]:
from lsst.sims.photUtils import BandpassDict

In [23]:
# Get the `bandpassDict` instance from files using catsim methods
lsstBP = BandpassDict.loadBandpassesFromFiles()
# This is a tuple, the first component gives the total bandpass, while the second gives the hardware bandpass

In [24]:
# Pass the bandpass and get the lightcurve for the observations 
reflc.lightCurve(idValue=6144007055260714, observations=df, bandPassDict=lsstBP[0])

,band,flux,fluxerr,m5,time
obsHistID,,,,,
2420278,u,7.199320e-12,8.797582e-11,23.364561,63187.066784
2421021,u,3.314851e-11,1.410967e-10,22.862402,63188.080403
2421812,u,7.690017e-11,1.404422e-10,22.868438,63189.117991
2423149,g,1.272596e-09,5.665873e-11,23.897282,63191.072483
2423169,i,2.148215e-09,9.308826e-11,23.357840,63191.082182
2423139,r,2.096949e-09,5.994450e-11,23.861980,63191.066950
2423215,y,9.526965e-10,4.722771e-10,21.564430,63191.104080
2423195,z,5.356918e-10,1.427617e-10,22.864327,63191.094381
2425238,g,9.330813e-09,9.346192e-11,23.423457,63194.058901


## Simpler methods for Twinkles  

For the Twinkles Data Analysis, we can use a simpler method. The full API is shown below:

In [25]:
reflcTwink = RefLightCurves.fromTwinklesData(tableName='TwinkSN',
                                             idCol='snid',
                                             objectTypeID=42,
                                             dbHostName=None,
                                             columns=('snid', 'redshift', 'snra', 'sndec', 't0',
                                                      'x0', 'x1', 'c'),
                                             idSequence=None)

The tableName, idCol, objectTypeID, and columns will change from one astrophysical object to another.
The defaults are currently correct for SN, so one could simply do
```
reflcTwink = RefLightCurves.fromTwinklesData(tableName='TwinkSN")
```

The light Curves can be accessed in the following way

In [26]:
reflcTwink.lightCurve(idValue=6144007055260714)

,band,flux,fluxerr,m5,time
obsHistID,,,,,
2420278,u,7.199320e-12,8.797582e-11,23.364561,63187.066784
2421021,u,3.314851e-11,1.410967e-10,22.862402,63188.080403
2421812,u,7.690017e-11,1.404422e-10,22.868438,63189.117991
2423149,g,1.272596e-09,5.665873e-11,23.897282,63191.072483
2423169,i,2.148215e-09,9.308826e-11,23.357840,63191.082182
2423139,r,2.096949e-09,5.994450e-11,23.861980,63191.066950
2423215,y,9.526965e-10,4.722771e-10,21.564430,63191.104080
2423195,z,5.356918e-10,1.427617e-10,22.864327,63191.094381
2425238,g,9.330813e-09,9.346192e-11,23.423457,63194.058901


We can get the light curves for each band by

In [27]:
reflcTwink.lightCurve(idValue=6144007055260714, bandName='r')

,band,flux,fluxerr,m5,time
obsHistID,,,,,
2423139,r,2.096949e-09,5.994450e-11,23.861980,63191.066950
2425228,r,8.633074e-09,8.818023e-11,23.509058,63194.053369
2431056,r,3.702634e-08,8.877849e-11,24.510950,63203.057703
2433141,r,4.211392e-08,9.445696e-11,24.463415,63206.061796
2435223,r,4.356894e-08,1.003223e-10,24.138431,63209.061903
2440811,r,3.098705e-08,8.168054e-11,24.557801,63219.118123
2442742,r,2.680369e-08,8.657722e-11,23.998963,63222.067830
